# 0. Introduction

# 1. Initialization

## 1.1. (most) package imports

In [1]:
# kaggle/python Docker image: https://github.com/kaggle/docker-python

import gc
import time
import os
import re
from contextlib import contextmanager
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 20, "display.max_columns", 30)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
import imblearn.pipeline
#from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

## 1.2. Global variables

In [2]:
offline = True

## 1.3. Global functions

In [3]:
@contextmanager
def timer(title):
    t0 = time.perf_counter()
    yield
    print("{} - done in {:.0f}s".format(title, time.perf_counter() - t0))

def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == "object"]
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

## 1.4. Dataset overview

offline j'ai:

<p style="background:black">
<code style="background:black;color:white">filenames = [
    "home-credit-default-risk/application_test.csv",
    "home-credit-default-risk/application_train.csv",
    "home-credit-default-risk/bureau.csv",
    "home-credit-default-risk/bureau_balance.csv",
    "home-credit-default-risk/credit_card_balance.csv",
    "home-credit-default-risk/installments_payments.csv",
    "home-credit-default-risk/POS_CASH_balance.csv",
    "home-credit-default-risk/previous_application.csv",
    "home-credit-default-risk/sample_submission.csv"
    ]
</code>
</p>

not offline (on Kaggle) j'ai:

<p style="background:black">
<code style="background:black;color:white">filenames = [
    "/kaggle/input/home-credit-default-risk/sample_submission.csv",
    "/kaggle/input/home-credit-default-risk/bureau_balance.csv",
    "/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv",
    "/kaggle/input/home-credit-default-risk/application_train.csv",
    "/kaggle/input/home-credit-default-risk/application_test.csv",
    "/kaggle/input/home-credit-default-risk/previous_application.csv",
    "/kaggle/input/home-credit-default-risk/credit_card_balance.csv",
    "/kaggle/input/home-credit-default-risk/installments_payments.csv",
    "/kaggle/input/home-credit-default-risk/bureau.csv"]
    ]
</code>
</p>

In [4]:
if offline:
    le_path = "home-credit-default-risk/"
else:
    le_path = "/kaggle/input"

In [5]:
filenames = []
for dirname, _, filenamess in os.walk(le_path):
    for filenamee in filenamess:
#                        HomeCredit_columns_description.csv est illisible.
        if filenamee != "HomeCredit_columns_description.csv":
            filename = os.path.join(dirname, filenamee)
#            print(filename)
            filenames.append(filename)
#            df = pd.read_csv(filename)
#            display(df[-1:])

In [6]:
if not offline:
    flnms = []
    flnms.append(filenames[4])
    flnms.append(filenames[3])
    flnms.append(filenames[8])
    flnms.append(filenames[1])
    flnms.append(filenames[6])
    flnms.append(filenames[7])
    flnms.append(filenames[2])
    flnms.append(filenames[5])
    flnms.append(filenames[0])
    filenames = flnms

# 2. Preprocessing

## 2.1. application_train and application_test

In [7]:
def application_train_test(num_rows=None, nan_as_category=False):
    df = pd.read_csv(filenames[1], nrows=num_rows)
    test_df = pd.read_csv(filenames[0], nrows=num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()

    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df["CODE_GENDER"] != "XNA"]

    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ["CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY"]:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)

    # Aberrant values
    df["DAYS_EMPLOYED"].replace(365243, np.nan, inplace=True)

    # Some simple new features (percentages)
    df["DAYS_EMPLOYED_PERC"] = df["DAYS_EMPLOYED"] / df["DAYS_BIRTH"]
    df["INCOME_CREDIT_PERC"] = df["AMT_INCOME_TOTAL"] / df["AMT_CREDIT"]
    df["INCOME_PER_PERSON"] = df["AMT_INCOME_TOTAL"] / df["CNT_FAM_MEMBERS"]
    df["ANNUITY_INCOME_PERC"] = df["AMT_ANNUITY"] / df["AMT_INCOME_TOTAL"]
    df["PAYMENT_RATE"] = df["AMT_ANNUITY"] / df["AMT_CREDIT"]

    del test_df
    gc.collect()
    return df

## 2.2. bureau and bureau_balance

In [8]:
def bureau_and_balance(num_rows=None, nan_as_category=True):
    bureau = pd.read_csv(filenames[2], nrows=num_rows)
    bb = pd.read_csv(filenames[3], nrows=num_rows)

    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)

    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {"MONTHS_BALANCE": ["min", "max", "size"]}
    for col in bb_cat:
        bb_aggregations[col] = ["mean"]
    bb_agg = bb.groupby("SK_ID_BUREAU").agg(bb_aggregations)
    bb_agg.columns = pd.Index(
        [e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()]
    )
    bureau = bureau.join(bb_agg, how="left", on="SK_ID_BUREAU")
    bureau.drop(["SK_ID_BUREAU"], axis=1, inplace=True)
    del bb, bb_agg
    gc.collect()

    # Bureau and bureau_balance numeric features
    num_aggregations = {
        "DAYS_CREDIT": ["min", "max", "mean", "var"],
        "DAYS_CREDIT_ENDDATE": ["min", "max", "mean"],
        "DAYS_CREDIT_UPDATE": ["mean"],
        "CREDIT_DAY_OVERDUE": ["max", "mean"],
        "AMT_CREDIT_MAX_OVERDUE": ["mean"],
        "AMT_CREDIT_SUM": ["max", "mean", "sum"],
        "AMT_CREDIT_SUM_DEBT": ["max", "mean", "sum"],
        "AMT_CREDIT_SUM_OVERDUE": ["mean"],
        "AMT_CREDIT_SUM_LIMIT": ["mean", "sum"],
        "AMT_ANNUITY": ["max", "mean"],
        "CNT_CREDIT_PROLONG": ["sum"],
        "MONTHS_BALANCE_MIN": ["min"],
        "MONTHS_BALANCE_MAX": ["max"],
        "MONTHS_BALANCE_SIZE": ["mean", "sum"]
    }

    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ["mean"]
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ["mean"]

    bureau_agg = bureau.groupby("SK_ID_CURR").agg(
        {**num_aggregations, **cat_aggregations}
    )
    bureau_agg.columns = pd.Index([
        "BURO_" + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()
    ])

    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau["CREDIT_ACTIVE_Active"] == 1]
    active_agg = active.groupby("SK_ID_CURR").agg(num_aggregations)
    active_agg.columns = pd.Index([
        "ACTIVE_" + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()
    ])
    bureau_agg = bureau_agg.join(active_agg, how="left", on="SK_ID_CURR")
    del active, active_agg
    gc.collect()

    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau["CREDIT_ACTIVE_Closed"] == 1]
    closed_agg = closed.groupby("SK_ID_CURR").agg(num_aggregations)
    closed_agg.columns = pd.Index([
        "CLOSED_" + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()
    ])
    bureau_agg = bureau_agg.join(closed_agg, how="left", on="SK_ID_CURR")
    del closed, closed_agg, bureau
    gc.collect()

    return bureau_agg

## 2.3. previous_applications

In [9]:
def previous_applications(num_rows=None, nan_as_category=True):
    prev = pd.read_csv(filenames[7], nrows=num_rows)

    prev, cat_cols = one_hot_encoder(prev, nan_as_category=True)

    # Aberrant values
    prev["DAYS_FIRST_DRAWING"].replace(365243, np.nan, inplace=True)
    prev["DAYS_FIRST_DUE"].replace(365243, np.nan, inplace=True)
    prev["DAYS_LAST_DUE_1ST_VERSION"].replace(365243, np.nan, inplace=True)
    prev["DAYS_LAST_DUE"].replace(365243, np.nan, inplace=True)
    prev["DAYS_TERMINATION"].replace(365243, np.nan, inplace=True)

    # Add feature: value ask / value received percentage
    prev["APP_CREDIT_PERC"] = prev["AMT_APPLICATION"] / prev["AMT_CREDIT"]

    # Previous applications numeric features
    num_aggregations = {
        "AMT_ANNUITY": ["min", "max", "mean"],
        "AMT_APPLICATION": ["min", "max", "mean"],
        "AMT_CREDIT": ["min", "max", "mean"],
        "APP_CREDIT_PERC": ["min", "max", "mean", "var"],
        "AMT_DOWN_PAYMENT": ["min", "max", "mean"],
        "AMT_GOODS_PRICE": ["min", "max", "mean"],
        "HOUR_APPR_PROCESS_START": ["min", "max", "mean"],
        "RATE_DOWN_PAYMENT": ["min", "max", "mean"],
        "DAYS_DECISION": ["min", "max", "mean"],
        "CNT_PAYMENT": ["mean", "sum"],
    }

    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ["mean"]

    prev_agg = prev.groupby("SK_ID_CURR").agg({**num_aggregations,
                                               **cat_aggregations})
    prev_agg.columns = pd.Index([
        "PREV_" + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()
    ])

    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev["NAME_CONTRACT_STATUS_Approved"] == 1]
    approved_agg = approved.groupby("SK_ID_CURR").agg(num_aggregations)
    approved_agg.columns = pd.Index([
        "APPROVED_" + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()
    ])
    prev_agg = prev_agg.join(approved_agg, how="left", on="SK_ID_CURR")

    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev["NAME_CONTRACT_STATUS_Refused"] == 1]
    refused_agg = refused.groupby("SK_ID_CURR").agg(num_aggregations)
    refused_agg.columns = pd.Index([
        "REFUSED_" + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()
    ])
    prev_agg = prev_agg.join(refused_agg, how="left", on="SK_ID_CURR")

    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

## 2.4. pos_cash

In [10]:
def pos_cash(num_rows=None, nan_as_category=True):
    pos = pd.read_csv(filenames[6], nrows=num_rows)

    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)

    # Features
    aggregations = {
        "MONTHS_BALANCE": ["max", "mean", "size"],
        "SK_DPD": ["max", "mean"],
        "SK_DPD_DEF": ["max", "mean"]
    }
    for cat in cat_cols:
        aggregations[cat] = ["mean"]

    pos_agg = pos.groupby("SK_ID_CURR").agg(aggregations)
    pos_agg.columns = pd.Index([
        "POS_" + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()
    ])

    # Count pos cash accounts
    pos_agg["POS_COUNT"] = pos.groupby("SK_ID_CURR").size()

    del pos
    gc.collect()
    return pos_agg

## 2.5. installment_payments

In [11]:
def installments_payments(num_rows=None, nan_as_category=True):
    ins = pd.read_csv(filenames[5], nrows=num_rows)

    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)

    # Features
    # Percentage and difference paid in each installment (amount paid and
    # installment value)
    ins["PAYMENT_PERC"] = ins["AMT_PAYMENT"] / ins["AMT_INSTALMENT"]
    ins["PAYMENT_DIFF"] = ins["AMT_INSTALMENT"] - ins["AMT_PAYMENT"]
    # Days past due and days before due (no negative values)
    ins["DPD"] = ins["DAYS_ENTRY_PAYMENT"] - ins["DAYS_INSTALMENT"]
    ins["DBD"] = ins["DAYS_INSTALMENT"] - ins["DAYS_ENTRY_PAYMENT"]
    ins["DPD"] = ins["DPD"].apply(lambda x: x if x > 0 else 0)
    ins["DBD"] = ins["DBD"].apply(lambda x: x if x > 0 else 0)

    # Features: Perform aggregations
    aggregations = {
        "NUM_INSTALMENT_VERSION": ["nunique"],
        "DPD": ["max", "mean", "sum"],
        "DBD": ["max", "mean", "sum"],
        "PAYMENT_PERC": ["max", "mean", "sum", "var"],
        "PAYMENT_DIFF": ["max", "mean", "sum", "var"],
        "AMT_INSTALMENT": ["max", "mean", "sum"],
        "AMT_PAYMENT": ["min", "max", "mean", "sum"],
        "DAYS_ENTRY_PAYMENT": ["max", "mean", "sum"]
    }
    for cat in cat_cols:
        aggregations[cat] = ["mean"]

    ins_agg = ins.groupby("SK_ID_CURR").agg(aggregations)
    ins_agg.columns = pd.Index([
        "INSTAL_" + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()
    ])

    # Count installments accounts
    ins_agg["INSTAL_COUNT"] = ins.groupby("SK_ID_CURR").size()

    del ins
    gc.collect()
    return ins_agg

## 2.6. credit_card_balance

In [12]:
def credit_card_balance(num_rows=None, nan_as_category=True):
    cc = pd.read_csv(filenames[4], nrows=num_rows)

    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)

    # General aggregations
    cc.drop(["SK_ID_PREV"], axis=1, inplace =True)

    cc_agg = cc.groupby("SK_ID_CURR").agg(["min", "max", "mean", "sum", "var"])
    cc_agg.columns = pd.Index([
        "CC_" + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()
    ])

    # Count credit card lines
    cc_agg["CC_COUNT"] = cc.groupby("SK_ID_CURR").size()

    del cc
    gc.collect()
    return cc_agg

# 3. functions from the original notebook

In [13]:
# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, num_folds, stratified=False, debug=False):
    # Divide in training/validation and test data
    train_df = df[df["TARGET"].notnull()]
    test_df = df[df["TARGET"].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(
        train_df.shape, test_df.shape
    ))
    del df
    gc.collect()

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits=num_folds,
                                shuffle=True,
                                random_state=1001)
    else:
        folds = KFold(n_splits=num_folds,
                      shuffle=True,
                      random_state=1001)

    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ["TARGET",
                                                      "SK_ID_CURR",
                                                      "SK_ID_BUREAU",
                                                      "SK_ID_PREV",
                                                      "index"]]

    for n_fold, (train_idx, valid_idx) in enumerate(
            folds.split(train_df[feats], train_df["TARGET"])
    ):
        train_x, train_y = train_df[feats].iloc[train_idx],\
                           train_df["TARGET"].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx],\
                           train_df["TARGET"].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1,
        )

        clf.fit(train_x,
                train_y,
                eval_set=[(train_x, train_y), (valid_x, valid_y)],
                eval_metric="auc",
                verbose=200,
                early_stopping_rounds=200)

        oof_preds[valid_idx] = clf.predict_proba(
            valid_x,
            num_iteration=clf.best_iteration_
        )[:, 1]
        sub_preds += clf.predict_proba(
            test_df[feats],
            num_iteration=clf.best_iteration_
        )[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat(
            [feature_importance_df, fold_importance_df],
            axis=0
        )
        print("Fold %2d AUC : %.6f" % (
            n_fold + 1,
            roc_auc_score(valid_y, oof_preds[valid_idx])
        ))

        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    # Write submission file and plot feature importance
    print("Full AUC score %.6f" % roc_auc_score(train_df["TARGET"], oof_preds))
    if not debug:
        test_df["TARGET"] = sub_preds
        test_df[["SK_ID_CURR", "TARGET"]].to_csv(submission_file_name,
                                                 index=False)
    display_importances(feature_importance_df)
    return feature_importance_df

In [14]:
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby(
        "feature"
    ).mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[
        feature_importance_df_.feature.isin(cols)
    ]

    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance",
                y="feature",
                data=best_features.sort_values(by="importance", ascending=False))
    plt.title("LightGBM Features (avg over folds)")
    plt.tight_layout()

    plt.savefig("lgbm_importances01.png")

# 4. Run the preprocessing

In [15]:
def preproc(debug=True):
    num_rows = debug if debug else None
    with timer("Process application train test"):
        df = application_train_test(num_rows)
        print("Application train test df shape:", df.shape)
        # print(df.dtypes.value_counts())
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how="left", on="SK_ID_CURR")
        # print(df.dtypes.value_counts())
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how="left", on="SK_ID_CURR")
        # print(df.dtypes.value_counts())
        del prev
        gc.collect()
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how="left", on="SK_ID_CURR")
        # print(df.dtypes.value_counts())
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how="left", on="SK_ID_CURR")
        # print(df.dtypes.value_counts())
        del ins
        gc.collect()
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how="left", on="SK_ID_CURR")
        # print(df.dtypes.value_counts())
        del cc
        gc.collect()

    zeros = df.TARGET.value_counts(
        sort=True,
        ascending=False,
        dropna=True,
    )[0]
    ones = df.TARGET.value_counts(
        sort=True,
        ascending=False,
        dropna=True,
    )[1]
    nans = df.TARGET.isna().sum()
    print("-----------------------------------------------------------------------")
    if debug:
        print("subsampled df's TARGET has",
              f"{zeros:10.0f} zeros,",
              f"{ones:10.0f} ones and",
              f"{nans:10.0f} NaNs")
    else:
        print("TARGET has",
              f"{zeros:10.0f} zeros,",
              f"{ones:10.0f} ones and",
              f"{nans:10.0f} NaNs")

    return zeros, ones, nans, df

## 4.1. A first full run just to measure the target imbalance

In [16]:
%%script echo
# I ran this cell only once, just to get the exact values of zo and oz.
if __name__ == "__main__":
    with timer("preproc_full"):
        zeros_full, ones_full, nans_full, df_full = preproc(debug=False)

Couldn't find program: 'echo'


In [17]:
zeros_full, ones_full, nans_full = 282682, 24825, 48744

In [18]:
zo = zeros_full/ones_full
oz = ones_full/zeros_full
print("There is zo =",
      f"{zo:.2f} more zeros than ones in TARGET. (and oz =",
      f"{oz:.2f})")

There is zo = 11.39 more zeros than ones in TARGET. (and oz = 0.09)


## 4.2. Subsampled run

### 4.2.1. Run the preprocessing

In [19]:
if __name__ == "__main__":
    with timer("preproc_subsampled"):
        zeros, ones, nans, df = preproc(debug=10_000)

Train samples: 10000, test samples: 10000
Application train test df shape: (20000, 246)
Process application train test - done in 1s
Bureau df shape: (2011, 108)
Process bureau and bureau_balance - done in 0s
Previous applications df shape: (9734, 242)
Process previous_applications - done in 0s
Pos-cash balance df shape: (9494, 15)
Process POS-CASH balance - done in 0s
Installments payments df shape: (8893, 26)
Process installments payments - done in 0s
Credit card balance df shape: (9520, 131)
Process credit card balance - done in 0s
-----------------------------------------------------------------------
subsampled df's TARGET has       9225 zeros,        775 ones and      10000 NaNs
preproc_subsampled - done in 2s


### 4.2.3. Measure of the target imbalance after the subsampling is done.

In [20]:
zo_sub = zeros/ones
oz_sub = ones/zeros
print("There is zo_sub =",
      f"{zo_sub:.2f} more zeros than ones in TARGET. (and oz_sub =",
      f"{oz_sub:.2f})")

There is zo_sub = 11.90 more zeros than ones in TARGET. (and oz_sub = 0.08)


In [21]:
if (zo/zo_sub >= 3/2 or zo/zo_sub <= 2/3):
    for _ in range(8):
        print("!")
    print(
        "L'imbalance des targets a été fortement modifiée par le subsampling.",
        f"Elle est passée de {zo:.2f} à {zo_sub:.2f}.",
    )
    for _ in range(8):
        print("!")
else:
    print(
        f"L'imbalance de {zo:.2f} des targets du dataset est de {zo_sub:.2f}",
        "après subsampling.",
    )

L'imbalance de 11.39 des targets du dataset est de 11.90 après subsampling.


# 6. Suppression du caractère illisible

In [22]:
cols_of_df = list(df.columns)

In [23]:
%%script echo skipped
# Ce code prend un temps infini à run. Prende la cell en-dessous.
for j in cols_of_df:
    df = df.rename(columns={j: re.sub(r"[ ]", r"_a_", j)})
    df = df.rename(columns={j: re.sub(r"[-]", r"_b_", j)})
    df = df.rename(columns={j: re.sub(r"[:]", r"_c_", j)})
    df = df.rename(columns={j: re.sub(r"[/]", r"_d_", j)})
    df = df.rename(columns={j: re.sub(r"[,]", r"_e_", j)})
    df = df.rename(columns={j: re.sub(r"[:]", r"_f_", j)})

Couldn't find program: 'echo'


In [24]:
%%script echo skipped
df = df.rename(columns=lambda x: x.replace(" ", "_a_")\
                                  .replace("-", "_b_")\
                                  .replace(":", "_c_")\
                                  .replace("/", "_d_")\
                                  .replace(",", "_e_")\
                                  .replace(":", "_f_"))

Couldn't find program: 'echo'


In [25]:
df = df.rename(columns=lambda x: x.replace(":", "deuxpoints"))

# 7. Classification run from the original notebook

In [26]:
%%script echo skipped
with timer("Run LightGBM with kfold"):
    feat_importance = kfold_lightgbm(df, num_folds=10, stratified=False, debug=False)

Couldn't find program: 'echo'


# 8. Classifications

## 8.0. Suppression des données sans TARGET
En effet mon but ici c'est juste de créer un modèle qui fonctionne, faire les
 prédictions de solvabilité des futurs clients ça sera pour plus tard.

In [27]:
df = df[df["TARGET"].notnull()]

## 8.1. train_test_split

In [28]:
X = df.drop("TARGET", axis="columns")
y = df["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [29]:
y_train.value_counts()

0.0    7405
1.0     595
Name: TARGET, dtype: int64

## 8.2. SimpleImputer

In [30]:
X_test_old = X_test.copy()

In [31]:
imputer = SimpleImputer(strategy="median")
#X_train_i = imputer.fit_transform(X_train)
X_train = imputer.fit_transform(X_train)
# X_test = imputer.fit_transform(X_test_old)
X_test = imputer.transform(X_test_old) # recommandé par mentor

## 8.3. Balancing the target

In [32]:
undersampler = RandomUnderSampler(sampling_strategy="majority")
#X_train_u, y_train_u = undersampler.fit_resample(X_train_i, y_train)

In [33]:
oversampler_1 = RandomOverSampler(sampling_strategy='minority')
oversampler_2 = SMOTE()
#X_train_o, y_train_o = oversampler.fit_resample(X_train_i, y_train)

## 8.4. Declaring the classifiers

In [34]:
clf0 = DummyClassifier()
clf1 = LogisticRegression(random_state=42)
clf2 = SGDClassifier()
clf3 = GaussianNB()
clf4 = MultinomialNB()
clf5 = KNeighborsClassifier()
clf6 = SVC(probability=True, random_state=42)
clf7 = DecisionTreeClassifier(random_state=42)
clf8 = RandomForestClassifier(random_state=42)
clf9 = GradientBoostingClassifier(random_state=42)
clf10 = AdaBoostClassifier(random_state=42)
clf11 = XGBClassifier(random_state=42)
clf12 = CatBoostClassifier(random_state=42)
clf13 = LGBMClassifier(random_state=42)

In [35]:
clfs = [
    clf0,
    clf1,
    clf2,
    clf3,
    clf4,
    clf5,
    clf6,
    clf7,
    clf8,
    clf9,
    clf10,
    clf11,
    clf12,
    clf13,
]

## 8.5. Declaring the classifiers' parameters

### 8.5.0. DummyClassifier

In [36]:
param0 = {}
param0["classifier__strategy"] = ["most_frequent",
                                  "prior"]
param0["classifier"] = [clf0]
#param0["classifier"] = [classifiers[0]]

### 8.5.1. LogisticRegression

In [37]:
param1 = {}
param1["classifier__C"] = [10**-2,
                           10**-1,
                           10**0,
                           10**1,
                           10**2]
param1["classifier__penalty"] = ["l1",
                                 "l2",
                                 "elasticnet"]
#param1["classifier__class_weight"] = [{0: 1, 1: 1},
#                                         {0: oz, 1: zo}]
#param1["classifier__class_weight"] = [None]
#param1["classifier__class_weight"] = [None,
#                                      {0: oz, 1: zo}]
# J'ai déjà fait un oversampling donc normalement je n'ai plus besoin de mettre
# les poids. Je teste juste ici.
param1["classifier"] = [clf1]
#param1["classifier"] = [classifiers[1]]

### 8.5.2. SGDClassifier

In [38]:
param2 = {}
param2["classifier__loss"] = ["hinge",
                              "log",
                              "squared_hinge",
                              "modified_huber"]
param2["classifier__penalty"] = ["l2",
                                 "l1",
                                 "elasticnet"]
param2["classifier"] = [clf2]
#param2["classifier"] = [classifiers[2]]

### 8.5.3. GaussianNB

In [39]:
param3 = {}
param3["classifier"] = [clf3]
#param3["classifier"] = [classifiers[3]]

### 8.5.4. MultinomialNB

In [40]:
param4 = {}
param4["classifier__alpha"] = [10**0,
                               10**1,
                               10**2]
param4["classifier"] = [clf4]
#param4["classifier"] = [classifiers[4]]

### 8.5.5. KNeighborsClassifier

In [41]:
param5 = {}
param5["classifier__n_neighbors"] = [10**.5,
                                     10**1,
                                     10**1.5,
                                     10**2]
param5["classifier__weights"] = ["uniform",
                                 "distance"]
param5["classifier"] = [clf5]
#param5["classifier"] = [classifiers[5]]

### 8.5.6. SVC

In [42]:
param6 = {}
param6["classifier__kernel"] = ["linear",
                                "rbf",
                                "poly",
                                "sigmoid"]
param6["classifier__C"] = [10**-2,
                           10**-1,
                           10**0,
                           10**1,
                           10**2,
                           10**3]
param6["classifier__gamma"] = ["auto",
                               "scale"]
#param6["classifier__class_weight"] = [None,
#                                         {0: oz, 1: zo}]
# J'ai déjà fait un oversampling donc plus besoin de mettre les poids.
#param6["classifier__class_weight"] = [None]
param6["classifier"] = [clf6]
#param6["classifier"] = [classifiers[6]]

### 8.5.7. DecisionTreeClassifier

In [43]:
param7 = {}
param7["classifier__max_depth"] = [3,
                                   10**1,
                                   30,
                                   None]
param7["classifier__min_samples_split"] = [10**.5,
                                           10**1]
#param7["classifier__class_weight"] = [{0: oz, 1: zo}]
# J'ai déjà fait un oversampling donc plus besoin de mettre les poids.
#param7["classifier__class_weight"] = [None]
param7["classifier"] = [clf7]
#param7["classifier"] = [classifiers[7]]

### 8.5.8. RandomForestClassifier

In [44]:
param8 = {}
param8["classifier__n_estimators"] = [10**1,
                                      10**2,
                                      10**3]
param8["classifier__max_depth"] = [3,
                                   10**1,
                                   30]
param8["classifier__criterion"] = ["gini",
                                   "entropy"]
#param8["classifier__class_weight"] = [None,
#                                         {0: oz, 1: zo}]
# J'ai déjà fait un oversampling donc plus besoin de mettre les poids.
#param8["classifier__class_weight"] = [None]
param8["classifier"] = [clf8]
#param8["classifier"] = [classifiers[8]]

### 8.5.9. GradientBoostingClassifier

In [45]:
param9 = {}
param9["classifier__n_estimators"] = [10**1,
                                      10**2,
                                      10**3]
param9["classifier__max_depth"] = [3,
                                   10**1,
                                   30]
param9["classifier"] = [clf9]
#param9["classifier"] = [classifiers[9]]

### 8.5.10. AdaBoostClassifier

In [46]:
param10 = {}
param10["classifier__n_estimators"] = [10**1,
                                       10**2,
                                       10**3]
param10["classifier__learning_rate"] = [10**-3,
                                        10**-2,
                                        10**-1]
param10["classifier"] = [clf10]
#param10["classifier"] = [classifiers[10]]

### 8.5.11. XGBClassifier

In [47]:
param11 = {}
param11["classifier__booster"] = ["gbtree",
                                  "gblinear",
                                  "dart"]
param11["classifier__learning_rate"] = [10**-3,
                                        10**-2,
                                        10**-1]
param11["classifier__max_depth"] = [10**0,
                                    3,
                                    10**1]
param11["classifier"] = [clf11]
#param11["classifier"] = [classifiers[11]]

### 8.5.12. CatBoostClassifier

In [48]:
param12 = {}
param12["classifier__iterations"] = [10**1,
                                     10**2,
                                     10**3]
param12["classifier__learning_rate"] = [10**-3,
                                        10**-2,
                                        10**-1]
param12["classifier__depth"] = [10**0,
                                3,
                                10**1]
param12["classifier"] = [clf12]
#param12["classifier"] = [classifiers[12]]

### 8.5.13. LGBMClassifier

In [49]:
param13 = {}
param13["classifier__boosting_type"] = ["gbdt",
                                        "dart",
                                        "goss"]
param13["classifier__learning_rate"] = [10**-3,
                                        10**-2,
                                        10**-1]
param13["classifier__num_leaves"] = [10**1,
                                     10**1.5,
                                     10**2]
param13["classifier"] = [clf13]
#param13["classifier"] = [classifiers[13]]

### 8.5bis. Déclaration des samplers

In [50]:
param_sampler = {}
param_sampler["sampler"] = [None,
                            undersampler,
                            oversampler_1,
                            oversampler_2]

### 8.5ter. Listing des paramètres

In [51]:
params_full = [
    param0,
    param1,
    param2,
    param3,
    param4,
    param5,
    param6,
    param7,
    param8,
    param9,
    param10,
    param11,
    param12,
    param13,
]

## 8.6. Itération sur les classifiers avec une boucle.
Je vais gridsearch les paramètres de chacun des estimateurs et enregistrer le
best_estimator_ obtenu à chaque fois pour pouvoir comparer les estimateurs.

Le problème c'est qu'ici apparemment on veut l'inverse. On veut plutôt random
entre tous les estimateurs pour trouver lequel est le meilleur et ne sortir que
ses stats à lui. Puis recommencer avec une autre technique de sampling.

In [52]:
sampler = SMOTE()
print(str(sampler)[:-2])

SMOTE


In [53]:
result_dfs = dict()
for sampler in param_sampler["sampler"]:
    # I create a different results table for each sampler.
    result_df = pd.DataFrame(columns=["classifier", "score"])
    # I gridsearch each classifier with its own set of parameters.
    for i, (clf, param) in enumerate(list(zip(clfs, params_full))):
        print(i)

        # Without sampler i still try to optimize the gridsearch using class_weight.
        if sampler is None:
            # class_weight does not exist for dummy, NBayes, KN, GB, AdaB, XGB
            if i not in [0, 3, 4, 5, 9, 10, 11]:
                param["classifier__class_weight"] = [None,
                                                     {0: .8*oz, 1: zo/.8},
                                                     {0: oz, 1: zo},
                                                     {0: oz/.8, 1: .8*zo},]

        # Train the model
        pipeline = imblearn.pipeline.Pipeline([("sampler", sampler),
                                               ("classifier", clf)])
        gs = GridSearchCV(
            pipeline,
            param,
            cv=5,
            # n_jobs=-1,
            scoring="roc_auc",
            # scoring=["roc_auc", "accuracy"],
            # refit="roc_auc",
            # n_iter=5, # seulement pour RandomSearchCV ?
            # random_state=42, # seulement pour RandomSearchCV ?
        ).fit(X_train, y_train)

        # Save the results
        print("Score:", gs.best_score_)
        print("Obtained with:", gs.best_params_["classifier"])
        result_df.append([gs.best_params_["classifier"], gs.best_score_])
    nom_du_df = "result_df_" + str(sampler)[:-2]
    result_df.to_csv(nom_du_df)
    result_dfs[str(sampler)] = result_df
    display(result_df[-2:])

0
Score: 0.5
Obtained with: DummyClassifier(strategy='most_frequent')
1


C:\Users\achou\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\achou\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
200 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error

Score: 0.5386560295961733
Obtained with: LogisticRegression(C=0.01, random_state=42)
2


C:\Users\achou\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\achou\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Score: 0.5366099444504338
Obtained with: SGDClassifier(class_weight={0: 0.07025562292611486, 1: 14.2337361530715},
              loss='squared_hinge', penalty='l1')
3
Score: 0.5676978421348282
Obtained with: GaussianNB()
4


C:\Users\achou\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\achou\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\achou\anaconda3\lib\site-packages\imblearn\pipeline.py", line 297, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "C:\Users\achou\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 690, in fit
    self._count(X, Y)
  File "C:\Users\achou\anaconda3\lib\site-

ValueError: Negative values in data passed to MultinomialNB (input X)

## 8.7. Itération sur les classifiers avec une méthode étrange
En fait ici on ne donne au SearchCV qu'une seule pipe de travail ne contenant
qu'un seul estimateur, mais il va quand même itérer sur tous les classifiers
car ils sont tous renseignés dans ```params```.

In [ ]:
params = [
    # DummyClassifier
    param0,
    # LogisticRegression
    # param1,
    # SGDClassifier
    # param2,
    # GaussianNB
    # param3,
    # MultinomialNB
    # param4,
    # KNeighborsClassifier
    param5,
    # SVC
    # param6,
    # DecisionTreeClassifier
    # param7,
    # RandomForestClassifier
    param8,
    # GradientBoostingClassifier
    # param9,
    # AdaBoostClassifier
    # param10,
    # XGBClassifier
    param11,
    # CatBoostClassifier
    # param12,
    # LGBMClassifier
    # param13,
    # Samplers
    # param_sampler,
]

In [ ]:
%%script echo
%%time
rs = RandomizedSearchCV(
    pipeline,
    params,
    n_iter=5,
    cv=3,
    # n_jobs=-1,
    scoring=["roc_auc", "accuracy"],
    refit="roc_auc",
    random_state=42
).fit(X_train, y_train)
#).fit(X_train_u, y_train_u)
#).fit(X_train_o, y_train_o)

## 8.8. Results

In [ ]:
rs.best_params_

In [ ]:
rs.best_score_

In [ ]:
y_test.value_counts(dropna=False)

In [ ]:
X_test.shape

In [ ]:
X_train_o.shape

In [ ]:
print("Precision:",
      precision_score(rs.predict(X_test), y_test))
      # precision_score(rs.best_estimator_.predict(X_test), y_test))
print("Recall:",
      recall_score(rs.best_estimator_.predict(X_test), y_test))
print("ROC AUC Score:",
      roc_auc_score(rs.best_estimator_.predict(X_test), y_test))

In [ ]:
%%script echo
#rs_cv.fit(X_train, y_train)
#rs_cv.fit(X_train_u, y_train_u)
rs_cv.fit(X_train_o, y_train_o)

for algorithm in classifiers.keys():
    print(f"Best parameters for {algorithm}: {rs_cv.best_params_[algorithm]}")
    print(f"Best AUC score for {algorithm}: {rs_cv.best_score_[algorithm]['roc_auc']:.3f}")
    print(f"Best accuracy score for {algorithm}: {rs_cv.best_score_[algorithm]['accuracy']:.3f}")

#best_algorithm = rs_cv.best_estimator_.keys()[0]
best_algorithm = rs_cv.best_estimator_.named_steps.keys()
print(f"Overall best algorithm: {best_algorithm}")
print(f"Best AUC score: {rs_cv.best_score_[best_algorithm]['roc_auc']:.3f}")
print(f"Best accuracy score: {rs_cv.best_score_[best_algorithm]['accuracy']:.3f}")

In [ ]:
rs.predict(X_test).shape

In [ ]:
y_test.shape

# 9. Fine tuning with the best classifier

## 9.1. GridSearchCV

## 9.2. Results

# 10. Applying the fine-tuned best classifier for the final results

In [ ]:
#y_pred = rs_cv.best_estimator_[best_algorithm].predict(X_test)

# 11. Features' importance

## 11.1. SHAP

## 11.2. LIME